In [1]:
from datasets import load_from_disk
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from transformers import DistilBertModel, DistilBertTokenizerFast

In [2]:
# Load the dataset from the specified directory
train_dataset = load_from_disk('DATA/ds_train')
valid_dataset = load_from_disk('DATA/ds_valid')

# Check the dataset structure
print(train_dataset)
print(valid_dataset)

Dataset({
    features: ['rating', 'text', 'labels', 'len_text', 'input_ids', 'attention_mask', 'n_tokens'],
    num_rows: 502341
})
Dataset({
    features: ['rating', 'text', 'labels', 'len_text', 'input_ids', 'attention_mask', 'n_tokens'],
    num_rows: 62793
})


In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)  # Set num_labels based on your task
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
training_args = TrainingArguments(
    output_dir="./results",        # Directory to save model checkpoints
    evaluation_strategy="steps",  # Evaluate every few steps
    eval_steps=10000,             # Evaluate every 10,000 steps
    save_strategy="epoch",        # Save model checkpoints at the end of each epoch
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",         # Directory to save TensorBoard logs
    logging_steps=100             # Log metrics every 100 steps
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_8134/2060051339.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
trainer.train()

Step,Training Loss,Validation Loss
10000,0.308900,0.292847
20000,0.282600,0.279046
30000,0.263000,0.283358
40000,0.210800,0.287177


TrainOutput(global_step=47097, training_loss=0.2623072730980322, metrics={'train_runtime': 3160.6937, 'train_samples_per_second': 476.801, 'train_steps_per_second': 14.901, 'total_flos': 3.390928083982098e+16, 'train_loss': 0.2623072730980322, 'epoch': 3.0})

In [10]:
trainer.evaluate()

{'eval_loss': 0.2849332094192505,
 'eval_runtime': 38.6388,
 'eval_samples_per_second': 1625.129,
 'eval_steps_per_second': 50.804,
 'epoch': 3.0}

In [24]:
test_dataset = load_from_disk('DATA/ds_test')

In [25]:
import torch
input_ids_tensor = torch.tensor(test_dataset['input_ids'][:3])

/tmp/ipykernel_8134/3752777607.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_tensor = torch.tensor(test_dataset['input_ids'][:3])


In [23]:
input_ids_tensor = torch.stack(test_dataset['input_ids'][:3])
attention_mask_tensor = torch.stack(test_dataset['attention_mask'][:3])

RuntimeError: stack expects each tensor to be equal size, but got [38] at entry 0 and [13] at entry 1

In [43]:
model(input_ids=test_dataset['input_ids'][:10].to("cuda:0"), attention_mask=test_dataset['attention_mask'][:10].to("cuda:0"))

SequenceClassifierOutput(loss=None, logits=tensor([[-3.2794, -3.6511,  5.0111],
        [-3.6481, -1.6206,  4.3256],
        [ 1.9507, -0.3739, -2.3655],
        [ 1.3998, -0.2083, -1.7181],
        [-0.2679, -0.0420,  0.2147],
        [-3.3602, -3.3848,  5.0193],
        [-2.6984,  0.0831,  2.1670],
        [-3.4293, -3.4148,  5.0653],
        [-3.5424, -3.2545,  5.1097],
        [-2.9918, -2.3719,  4.2830]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [42]:
test_dataset['labels'][:10]

tensor([2, 2, 2, 0, 0, 2, 2, 2, 2, 2])

In [52]:
test_dataset['text']

['this is a great deal on awesome scrunchies.',
 'perfect of the bottle. i like the pump, it’s working very well and 16 oz bottle. it is very good sides for soap dishes .',
 'it literally tasted like i was putting hydro peroxide and didn’t really do much, i returned.',
 'nail polish just peels off',
 "the colors where to bright didn't expect them that color.",
 'it’s exactly what i wanted and the cheapest one i could find after ulta discontinued theirs. i originally bought one from ulta for my mom and fell and love with hers so i ordered this one off of amazon and it’s exactly the same. came in new condition with all of the plastic still on. thank you!!',
 "this item is great but it doesn't stay warm very long. it is very warm after microwaving but cools down much faster than i expected since it states to keep on for 15-20 minutes. it stays warm for approximately 8 minutes. i timed it. but does feel great and adjusts well.",
 "absolutely love these! i've never worn false lashes before 

In [54]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

In [66]:
input_ids = test_dataset['input_ids'].to("cuda:0")
attention_mask = test_dataset['attention_mask'].to("cuda:0")
labels = test_dataset['labels'].to("cuda:0")
eval_batch_size = 128
current_positon = 0
is_stop = False

output = []
while True:
    end_position = current_positon + eval_batch_size
    if end_position > n_examples:
        end_position = n_examples
        is_stop = True
    output.append(model(input_ids=input_ids[current_positon:end_position], attention_mask=attention_mask[current_positon:end_position]))
    current_positon = end_position
    if is_stop:
        break

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 9.77 GiB of which 13.94 MiB is free. Process 1928571 has 9.75 GiB memory in use. Of the allocated memory 9.31 GiB is allocated by PyTorch, and 178.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [57]:
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Data collator automatically pads sequences in batches
data_collator = DataCollatorWithPadding(tokenizer, return_tensors="pt")

# Create a DataLoader
dataloader = DataLoader(
    test_dataset[:50],  # Assuming test_dataset is tokenized and formatted
    batch_size=10,  # Define the batch size
    collate_fn=data_collator
)

# Iterate through the DataLoader
for batch in dataloader:
    # Move inputs to cuda:0
    input_ids = batch["input_ids"].to("cuda:0")
    attention_mask = batch["attention_mask"].to("cuda:0")
    
    # Forward pass
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Access logits or other outputs
    logits = outputs.logits
    print(logits)

KeyError: 0

In [67]:
import torch
torch.cuda.empty_cache()
